# Imports

In [1]:
import numpy as np

import os
import json
import time

import torch
from torch import nn
import torch.nn.functional as F

# check if notebook is in colab
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass

import ezkl

In [2]:
print(ezkl.__version__, flush=True)

22.2.1


In [3]:
!lscpu | grep "Model name:" | sed -r 's/Model name:\s{1,}//g'

Model name:                           12th Gen Intel(R) Core(TM) i5-1235U


# Setup paths for ML models

In [21]:
model_folders = [f for f in os.listdir("../testdata/") if not f.startswith(".")]
model_folders.sort()

run_args = ezkl.PyRunArgs()
run_args.variables = [("batch_size", 1)]

print(model_folders, flush=True)

['2d_conv', 'lenet_5', 'mobilenet', 'xg_boost']


# EZKL (for each model)

In [22]:
# filter models if you want :)
model_folders = [model_folders[3]]
print(model_folders, flush=True)

['xg_boost']


In [23]:
for model_folder in model_folders:
    print(f"Model: {model_folder}", flush=True)

    model_folder = f"../testdata/{model_folder}"

    os.makedirs(model_folder, exist_ok=True)

    # set variables
    model_path = f"{model_folder}/network.onnx"
    data_path = f"{model_folder}/input.json"
    settings_path = f"{model_folder}/settings.json"
    cal_data_path = f"{model_folder}/cal_input.json"
    compiled_model_path = f"{model_folder}/network.compiled"
    srs_path = f"{model_folder}/network.srs"
    witness_path = f"{model_folder}/witness.json"
    vk_path = f"{model_folder}/network.vk"
    pk_path = f"{model_folder}/network.pk"
    proof_path = f"{model_folder}/proof.pf"

    # generate settings
    print("Generating settings...", flush=True)
    start_time = time.time()
    res = ezkl.gen_settings(model_path, settings_path, py_run_args=run_args)
    end_time = time.time()
    print(f"Time taken to generate settings: {end_time - start_time} seconds", flush=True)
    assert res == True

    # calibrate settings
    data = json.load(open(data_path, "r"))
    cal_data = np.array(data["input_data"]).flatten()
    num_elements = cal_data.size
    random_data = (torch.rand(num_elements, requires_grad=True).detach().numpy()).reshape([-1]).tolist()
    json.dump(dict(input_data = [(torch.rand(num_elements, requires_grad=True).detach().numpy()).reshape([-1]).tolist()]), open(cal_data_path, 'w'))

    print("Skipping calibrating settings due to some problems...", flush=True)
    # start_time = time.time()
    # res = ezkl.calibrate_settings(cal_data_path, model_path, settings_path, "resources")
    # end_time = time.time()
    # print(f"Time taken to calibrate settings: {end_time - start_time} seconds")
    # assert res == True

    # compile circuit
    print("Compiling circuits...", flush=True)
    start_time = time.time()
    res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
    end_time = time.time()
    print(f"Time taken to compile circuit: {end_time - start_time} seconds", flush=True)
    assert res == True

    # get srs
    print("Getting srs...", flush=True)
    start_time = time.time()
    res = await ezkl.get_srs(settings_path, srs_path=srs_path)
    end_time = time.time()
    print(f"Time taken to get srs: {end_time - start_time} seconds", flush=True)
    assert res == True

    # generate witness
    print("Generating witness...", flush=True)
    start_time = time.time()
    res = ezkl.gen_witness(data_path, compiled_model_path, witness_path, srs_path=srs_path)
    end_time = time.time()
    print(f"Time taken to generate witness: {end_time - start_time} seconds", flush=True)
    assert os.path.isfile(witness_path)

    # setup (create keys)
    print("Setting up keys...", flush=True)
    start_time = time.time()
    res = ezkl.setup(compiled_model_path, vk_path, pk_path, srs_path=srs_path)
    end_time = time.time()
    print(f"Time taken to setup keys: {end_time - start_time} seconds", flush=True)
    assert res == True

    # generate proof
    print("Generating proof...", flush=True)
    start_time = time.time()
    res = ezkl.prove(witness_path, compiled_model_path, pk_path, proof_path, "single", srs_path=srs_path)
    end_time = time.time()
    print(f"Time taken to generate proof: {end_time - start_time} seconds", flush=True)
    assert os.path.isfile(proof_path)

    # verify proof
    print("Verifying proof...", flush=True)
    start_time = time.time()
    res = ezkl.verify(proof_path, settings_path, vk_path, srs_path=srs_path)
    end_time = time.time()
    print(f"Time taken to verify proof: {end_time - start_time} seconds", flush=True)
    assert res == True

    print("Done!", flush=True)

    break

Model: xg_boost
Generating settings...
Time taken to generate settings: 0.04213523864746094 seconds
Skipping calibrating settings due to some problems...
Compiling circuits...


low scale values (<8) may impact precision


Time taken to compile circuit: 0.008542537689208984 seconds
Getting srs...
Time taken to get srs: 6.003355503082275 seconds
Generating witness...
Time taken to generate witness: 0.03211402893066406 seconds
Setting up keys...
Time taken to setup keys: 14.713239192962646 seconds
Generating proof...
Time taken to generate proof: 22.297100067138672 seconds
Verifying proof...
Time taken to verify proof: 0.08801579475402832 seconds
Done!
